### copying the code from previous exercise

In [1]:
using Pkg

#Pkg.activate("path/to/project")
Pkg.activate("~/CodeProjects/RosenbluthChains/")
###julia ]; activate ; dev --local PathToRosenbluthPackage
#Pkg.instantiate()
using RosenbluthChains, Plots

  Activating project at `~/Code_Projects/rosenbluthchains/~/CodeProjects/RosenbluthChains`


In [ ]:
### type definition
mutable struct My_Measurement{T<:Real} <: AbstractMeasurement 
    x::Vector{T}
    y::Vector{T}
    z::Vector{T}
end
My_Measurement(N::Int) = My_Measurement{Float64}(zeros(N), zeros(N), zeros(N)) ### constructs a Measurement 

### initialise measurments with empty vectors
function RosenbluthChains.InitMeasurement(data::SimData, param::SimulationParameters, Tmp::My_Measurement) 
    return My_Measurement(data.NBeads)
end 


### does nothing
function RosenbluthChains.MeasureAfterBatch(data::SimData, param::SimulationParameters, Measurement::My_Measurement) 
    nothing
end

function RosenbluthChains.MeasureAfterChainGrowth(data::SimData, param::SimulationParameters,Measurement::My_Measurement) 
    Measurement.x = getindex.(data.xyz, 1) ### gets the first element for each element of data.xyz
    Measurement.y = getindex.(data.xyz, 2) ### gets the second element for each element of data.xyz
    Measurement.z = getindex.(data.xyz, 3) ### gets the third element for each element of data.xyz
end

### does nothing
function SaveMeasurements(data::SimData, param::SimulationParameters,Measurement::My_Measurement) 
    nothing
end


#https://hoomd-blue.readthedocs.io/en/v4.7.0/module-md-angle.html#hoomd.md.angle.Harmonic
mutable struct HarmonicBondLength{T<:Real} <: RosenbluthChains.AbstractBondParam 
    k  ::T ### bond strength for all particles
    r0 ::T ### minima of the bond length potential
    Δr ::T ### sampling width=> take values from interval [r0-Δr, r0+Δr]
end
HarmonicBondLength(k ::T, r0::T, Δr::T) where {T<:Real} = HarmonicBondLength{T}(k, r0, Δr) ### constructs a Measurement 

@inline function SetTrialRadius(data::SimData, param::HarmonicBondLength)
   rand!(data.trial_radius, eltype(data.TType)) ### created NTrial many random number ∈ [0,1]
   data.trial_radius .*= 2.0*param.Δr ### ∈ [0, 2Δr]
   data.trial_radius .+= param.r0-param.Δr ### ∈ [r0-Δr, r0+Δr]
   return nothing
end


@inline function GetTrialBoltzmannWeight(data::SimData,param::HarmonicBondLength) 
    fill!(data.tmp4, 0.0) ### temporary array of length NTrials that i will use to store the energy

    data.tmp4  .= @. 1.0/2.0*(data.trial_radius-r0)^2 ### compute energies

    ### here comes a numerical trick to stabilise, we add the energies of al potentials first and take the exponential in "GetTrialBoltzmannWeight" before we have to choose the positions in "ChooseTrialPosition"
    # -= since we have to add all energies and take exp(-kt*E); kt=1
    data.LogBoltzmannFaktor .-= data.tmp4
    return nothing
end

### Exercise: Sampling Width in the Harmonic Potential
What happens when fix the value for bond strength k while varying the sampling with Δr? How would you choose Δr and why? Thinking about the tests from 1_Radius.ipynb might help.


### Exercise: Rosenbluth Weight
Compute the Rosenbluth for an ideal chain with random bond angles and dihedral angles with an harmonic bond potential. How does the Rosenbluth weight compare for a chain where each bond is exactly r0 compared to our method where we get variance for the bond distance. ? How do these weights differ for varying polymer length N? 

### Advanced Exercise/ Discussion: More Efficiency
Can generate we generate polymer chains in a way such that the weight does not decay with the length of the polymer. In other works can be sample close to the minima of the potentials such that we do an importance sampling where the important configurations are generate with high weights quicker. This would enhance numerical stability and sample more efficiently. 

Optional: Implement a method that would improve the sampling of an harmonic bond.